# Processamento de Imagens

Foi utilizado o processamento de imagens para rápida e automaticamente detectar as bordas dos riscos observados sob microscopia óptica. Este processamento gerou como resultados os valores de largura de desgaste utilizados posteriormente na análise estatística.

## Bibliotecas Utilizadas

Além das bibliotecas `numpy`, `pandas` e `plotly`, para tratamento e visualização de dados, foi utilizada a biblioteca `cv2` para manipular as imagens e obter representações matemáticas destas. Com essa representação matemática, é relativamente simples detectar as regiões da imagem.

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from cv2 import imread, imwrite
from ipywidgets import interact, widgets
from pathlib import Path
from os import listdir
from os.path import isfile, join

## Importação das Imagens

As imagens armazenadas na pasta `scr/Riscos_umidade2` são lidas e armazenadas em um dicionário `imgs` cujas chaves são o nome do arquivo e os valores são uma representação numérica do conteúdo da imagem.

### Representação da Imagem

As imagens são lidas como o tipo `np.ndarray`, ou seja, uma matriz tridimensional de inteiros. Cada inteiro representa a intensidade de uma cor primária (vermelho, verde ou azul) em um pixel da imagem. A imagem é, então, formada pela matriz de dimensões que correspondem a largura, altura e cor da imagem. Neste caso, as imagens possuem 2048 pixels de largura e 1532 pixels de altura. Portanto, a matriz possui ordem $1532 \times 2048 \times 3$.

In [2]:
p = Path('src/Riscos_umidade')

In [3]:
filenames = [f for f in listdir(p) if isfile(join(p, f))]
imgs = dict()

for filename in filenames:
    filepath = p/filename
    try:
        imgs[filepath.name] = imread(str(filepath))
    except FileNotFoundError:
        pass

## Tratamento dos Dados

Cada imagem foi armazenada em um DataFrame para facilitar a interpretação dos resultados. Cada coluna do DataFrame armazena uma cor, e cada linha armazena a soma do valor numérico em uma linha de pixels da imagem. Uma vez que os riscos estão alinhados horizontalmente à imagem, essa soma representa muito bem uma média de onde começa e onde acaba o risco na imagem.

![Imagem-Soma](src/risco_soma.png)

Definidas essas somas, foi escolhido um dos canais (vermelho) para se realizar a detecção das bordas, por ser o mais alto dentre os três canais de cor. No programa [06 - Image Processing Testing](06-Image_Processing-Testing_Old_Images.ipynb) foi observado que o melhor limiar para definir o início e final do risco é o de 85% da média entre o maior e o menor valor de cor. Ou seja, quando a intensidade de cor é 87,3% da intensidade média a leitura se refere à margem do risco.

A largura em pixels entre as duas margens é calculada, e o valor é traduzido para micrometros a partir da calibração do microscópio (cada pixel representa 1,26576 µm).

In [4]:
dfs = dict()
left_edge_px = dict()
right_edge_px = dict()
high = dict()
low = dict()
width = dict()

for key, img in imgs.items():
    dfs[key] = pd.DataFrame({'Blue': imgs[key][:,:,0].sum(axis = 1),
                       'Red': imgs[key][:,:,2].sum(axis = 1),
                       'Green': imgs[key][:,:,1].sum(axis = 1)})
    high[key] = max(dfs[key]['Blue'])
    low[key] = min(dfs[key]['Blue'])
    
for key in list(imgs.keys()):
    left_edge_px[key] = dfs[key][dfs[key]['Blue']>=0.873*np.mean((high[key],low[key]))].index.min()
    right_edge_px[key] = dfs[key][dfs[key]['Blue']>=0.873*np.mean((high[key],low[key]))].index.max()
    width[key] = 1.26576*(right_edge_px[key] - left_edge_px[key])

## Gráficos

Aqui são definidos dois gráficos. O primeiro mostra o nível de cada cor ao longo da imagem, considerando as somas realizadas no passo anterior. Foi adicionada uma anotação para mostrar onde foi considerado o limiar de detecção da margem do risco. O segundo gráfico mostra a imagem e as posições das margens detectadas, mostrando que a detecção é bem-sucedida.

In [5]:
def plot_colors(key):
    fig = px.line(dfs[key])
    fig.add_hline(y = 0.85*np.mean((high[key],low[key])), line_color = 'black',
              annotation_text="Limiar de Detecção", 
              annotation_position="bottom right")
    fig.show()

In [21]:
def plot_overlay(key):
    #path = p/key
    # Create figure
    fig = go.Figure()

    # Constants
    img_width = 2048
    img_height = 1532
    scale_factor = 0.3

    # Add invisible scatter trace.
    # This trace is added to help the autoresize logic work.
    fig.add_trace(
        go.Scatter(
            x=[0, img_width * scale_factor],
            y=[0, img_height * scale_factor],
            mode="markers",
            marker_opacity=0
        )
    )

    # Configure axes
    fig.update_xaxes(
        visible=False,
        range=[0, img_width * scale_factor]
    )

    fig.update_yaxes(
        visible=False,
        range=[0, img_height * scale_factor],
        # the scaleanchor attribute ensures that the aspect ratio stays constant
        scaleanchor="x"
    )

    # Add image
    fig.add_layout_image(
        dict(
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source = 'https://drive.google.com/uc?export=view&id=1c5gGfB03VJXPe5OxjUvFJuzqIPJ9WGIg')
            #source='src/riscos_umidade/{}'.format(key)[:-4]+'.jpg')
        )
    #fig.add_hline(y = 0.3*1532-0.3*left_edge_px[key], line_color = 'red')
    #fig.add_hline(y = 0.3*1532-0.3*right_edge_px[key], line_color = 'red')

    # Configure other layout
    fig.update_layout(
        width=img_width * scale_factor,
        height=img_height * scale_factor,
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )

    # Disable the autosize on double click because it adds unwanted margins around the image
    # More detail: https://plotly.com/python/configuration-options/
    fig.show(config={'doubleClick': 'reset'})

Cada gráfico se refere aos valores de uma imagem. Veja que para todas as amostras, exceto as de PU+4%PAni (PP4), são observados limites bem definidos do risco. Essas amostras foram as que apresentaram bolhas e a medida da largura não foi possível, como pode se ver no próximo gráfico.

In [22]:
plot_overlay(0)

In [7]:
interact(plot_colors, key = list(dfs.keys()))

interactive(children=(Dropdown(description='key', options=('GP4-01a.jpg', 'GP4-01b.jpg', 'GP4-01c.jpg', 'GP4-0…

<function __main__.plot_colors(key)>

In [8]:
interact(plot_overlay, key = list(imgs.keys()))

interactive(children=(Dropdown(description='key', options=('GP4-01a.jpg', 'GP4-01b.jpg', 'GP4-01c.jpg', 'GP4-0…

<function __main__.plot_overlay(key)>

## Resultados

As larguras medidas são salvas no dicionário `width`, que será organizado em um DataFrame com as informações de material e número de ensaio e exportado para o arquivo `data_largura.csv` que será juntado aos dados de coeficiente de atrito e umidade/temperatura posteriormente.

In [9]:
width

{'GP4-01a.jpg': 503.77248,
 'GP4-01b.jpg': 493.64639999999997,
 'GP4-01c.jpg': 497.44368,
 'GP4-02a.jpg': 387.32256,
 'GP4-02b.jpg': 383.52528,
 'GP4-02c.jpg': 392.3856,
 'GP4-03a_ne.jpg': 372.13344,
 'GP4-03b.jpg': 372.13344,
 'GP4-03c.jpg': 379.728,
 'GP4-04a.jpg': 286.06176,
 'GP4-04b.jpg': 293.65632,
 'GP4-04c.jpg': 294.92208,
 'GP4-05a.jpg': 373.3992,
 'GP4-05b.jpg': 369.60192,
 'GP4-05c.jpg': 370.86768,
 'GP4-06a.jpg': 372.13344,
 'GP4-06b.jpg': 373.3992,
 'GP4-06c.jpg': 372.13344,
 'GP4-07a.jpg': 373.3992,
 'GP4-07b.jpg': 369.60192,
 'GP4-07c.jpg': 372.13344,
 'GP4-08a.jpg': 444.28176,
 'GP4-08b.jpg': 441.75024,
 'GP4-08c.jpg': 446.81328,
 'GP4-09a.jpg': 268.34112,
 'GP4-09b.jpg': 268.34112,
 'GP4-09c.jpg': 270.87264,
 'GP4-10a.jpg': 346.81824,
 'GP4-10b.jpg': 345.55248,
 'GP4-10c.jpg': 345.55248,
 'GP4-11a.jpg': 372.13344,
 'GP4-11b.jpg': 367.0704,
 'GP4-11c.jpg': 375.93072,
 'GP4-12a.jpg': 254.41776,
 'GP4-12b.jpg': 237.96288,
 'GP4-12c.jpg': 239.22864,
 'GP4-13a.jpg': 273.404

In [11]:
df = pd.DataFrame()
for key, value in width.items():
    material, ensaio = key.split('-')
    ensaio = ensaio.split('_')[0].split('.')[0]
    df = df.append(pd.DataFrame([[material, ensaio[:2],value, key]], columns = ['Material', 'Ensaio', 'Largura', 'Arquivo']), ignore_index = True)

In [12]:
df.to_csv('src/data_largura.csv', index = False)